In [1]:
"""
This is the sentiment analysis with

POS tagging

"""

In [2]:
import os
import nltk
import random
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.text import Text
from nltk.classify import ClassifierI
import string, re
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

documents = []
all_words = []
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
allowed_pos = ['J', 'R']

In [3]:
import io
import re

yelp = io.open("all_reviews.txt", "r", encoding = 'utf-8_sig')
pos = {}
neg = {}

reviews = yelp.readlines()
reviews = [x.strip() for x in reviews] # seperate each line
star_flag = bool(False)
star = 0
documents = []
temp = ''
for line in reviews:
    if line == '{{{':
        star_flag = bool(True)
        continue
        
    if star_flag:
        star = (int(re.findall(r'^[1-5]',line)[0]))
        star_flag = bool(False)
        continue
        
    if line == '':
        continue
        
    if line == '}}}':
        continue
        
    if line == '[[[':
        continue
        
    else:
        if line == ']]]':
            temp = temp.replace('\xa0','') #some wierd encoding error stuff...
            words = tokenizer.tokenize(temp)
            
            #determine if the review is negative or positive and append it to documents list
            if star > 3:
                documents.append((words, 'pos'))
            else:
                documents.append((words, 'neg'))
            
            #add the tokenized words to bag of words
            tagged_words = nltk.pos_tag(words)
            for word in tagged_words:
                if word[1][0] in allowed_pos:
                    all_words.append(word[0])
            
            #reset temp
            temp = ''
        else:
            temp += line


In [4]:
#function that finds features from a document
#features are top 20% most frequent words from all documents

all_words = nltk.FreqDist(all_words)
top_10 = int(len(all_words)*.20)
word_features, frequency = map(list, zip(*all_words.most_common(top_10)))

def find_features(document):
    words = set(document)
    features = {}
    for word in word_features:
        features[word] = (word in words)
        
    return features

In [5]:
featuresets = [(find_features(review), category) for (review, category) in documents]
random.shuffle(featuresets)
print(len(featuresets))
first75 = int(len(featuresets)*.75)

training_set = featuresets[:first75]
testing_set = featuresets[first75:]

10391


In [6]:
names = ['MultinomialNB','BernoulliNB', 
         'Decision_Tree',
         'Logistic Regression',
         'SVC', 'LinearSVC', 'NuSVC']

trained = []

classifiers = [MultinomialNB(), BernoulliNB(), 
               DecisionTreeClassifier(random_state=0), 
               LogisticRegression(),
               SVC(), LinearSVC(), NuSVC()]

for name, clf in zip(names, classifiers):
    print('Now classifying', name)
    
    classifier = SklearnClassifier(clf)
    classifier.train(training_set)
    trained.append(classifier)
    print("{} classifier accuracy:".format(name), nltk.classify.accuracy(classifier, testing_set))
    print("\n")

Now classifying MultinomialNB
MultinomialNB classifier accuracy: 0.8371824480369515


Now classifying BernoulliNB
BernoulliNB classifier accuracy: 0.74788298691301


Now classifying Decision_Tree
Decision_Tree classifier accuracy: 0.7906081601231717


Now classifying Logistic Regression


C:\Users\YunMo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Logistic Regression classifier accuracy: 0.8371824480369515


Now classifying SVC
SVC classifier accuracy: 0.8602771362586605


Now classifying LinearSVC


C:\Users\YunMo\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC classifier accuracy: 0.8290993071593533


Now classifying NuSVC
NuSVC classifier accuracy: 0.859122401847575


